<a href="https://colab.research.google.com/github/Brandon-Yip2/AndroidStudioProjectCollection/blob/main/MultimodalFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocessing for text

In [ ]:
from google.colab import drive
import os

In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
!unzip Episode_transcripts.zip -d EpisodeTranscripts

unzip:  cannot find or open Episode_transcripts.zip, Episode_transcripts.zip.zip or Episode_transcripts.zip.ZIP.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')

%cd EpisodeTranscripts/Episode\ transcipts

episode_list = ["01a.txt", "01c.txt", "02a.txt", "02b.txt", "03a.txt", "03b.txt",
                "04a.txt", "04b.txt", "05a.txt", "05b.txt", "06a.txt", "06b.txt",
                "07a.txt", "07b.txt", "08a.txt", "08b.txt", "09a.txt", "09b.txt",
                "10a.txt", "10b.txt", "11a.txt", "11b.txt", "12a.txt", "12b.txt",
                "13a.txt", "13b.txt", "14a.txt", "14b.txt", "15a.txt", "15b.txt",
                "16a.txt", "16b.txt", "17a.txt", "17b.txt", "18a.txt", "18b.txt",
                "19a.txt", "19b.txt", "20a.txt", "20b.txt"
                ]
cols = ['text', 'words', 'id']
lst = []
index = 0
print(os.getcwd())

for filename in episode_list:
  f = open(filename, 'r')

  file_contents = f.read()

  # print (file_contents)
  # nltk.download('stopwords')


  f.close()


  stop_words = set(stopwords.words('english'))


  # Expand contractions
  # file_contents = file_contents.apply(lambda x: [contractions.fix(word) for word in x.split()])
  file_contents = file_contents.lower()
  file_contents = contractions.fix(file_contents)


  file_contents = re.sub("[\n]", ' ', file_contents)
  file_contents = re.sub("[^ 0-9a-zA-Z]", '', file_contents)
  # print(file_contents)

  file_contents = [w for w in file_contents.split() if not w in stopwords.words('english')]
  # print(file_contents)

  # Stemming text (reduce vocabulary size, let model treat word variations as same word)
  # the stemmer requires a language parameter
  snow_stemmer = SnowballStemmer(language='english')

  #stem's of each word
  stem_words = []
  for w in file_contents:
      x = snow_stemmer.stem(w)
      stem_words.append(x)

  # print(stem_words)
  # #print stemming results
  # for e1,e2 in zip(file_contents,stem_words):
  #     print(e1+' ----> '+e2)

  lst.append([filename, stem_words, index])
  index = index+1;

df1 = pd.DataFrame(lst, columns=cols)
print(df1)

%cd ../..
testlst = []
#Get the words detected from the audio files
df_ReadAudio = pd.read_csv('normalTranscripts.csv')

for ind in df_ReadAudio.index:

    test_contents = df_ReadAudio['text'][ind]
    test_contents = test_contents.lower()
    test_contents = contractions.fix(test_contents)
    test_contents = re.sub("[\n]", ' ', test_contents)
    test_contents = re.sub("[^ 0-9a-zA-Z]", '', test_contents)
    test_contents = [w for w in test_contents.split() if not w in stopwords.words('english')]
    test_stem_words = []
    for w in test_contents:
        x = snow_stemmer.stem(w)
        test_stem_words.append(x)


    col = ["testwords", "id"]
    testlst.append([test_stem_words, df_ReadAudio['episodeNumber'][ind]])
    df2 = pd.DataFrame(testlst, columns=col)

print(df2)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[Errno 2] No such file or directory: 'EpisodeTranscripts/Episode transcipts'
/content
/content


FileNotFoundError: [Errno 2] No such file or directory: '01a.txt'

Feature extraction for texts

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df1["words"]

In [ ]:
# Get word counts in data
word2count = {}

for row in df1["words"]:
    for word in row:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

# Constrain word count to top 1000 most popular words
# word2count_frequent = sorted(word2count.items(),  key=lambda x:x[1], reverse=True)[0:1000]
# Drop word counts from result
# unique_words = [i[0] for i in word2count_frequent]
# unique_words = list(word2count.keys())
print(len(word2count))


In [ ]:
# TF-IDF Vectorizer requires dummy function for tokenizer and preprocessor because text is already tokenized (just return the input)
def dummy_fun(doc):
    return doc

# Compute TF-IDF
tr_idf_model = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    max_features=1000) # Take top 1000 features

tf_idf_vector = tr_idf_model.fit_transform(df1["words"])
tf_idf_array = tf_idf_vector.toarray()

# Use training data vocabulary for test data vectorizer so that word features are in the same position
tr_idf_model_test = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    vocabulary=tr_idf_model.get_feature_names_out()) # use vocabulary from training data


tf_idf_vector_test = tr_idf_model_test.fit_transform(df2["testwords"])
tf_idf_array_test = tf_idf_vector_test.toarray()

print(tf_idf_array.shape)
print(len(tf_idf_array[0]))

print(tf_idf_array_test.shape)
print(tf_idf_array_test[0])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, roc_curve, auc, classification_report

In [ ]:
# Define all my classifiers
lc = LogisticRegression(max_iter=1000)
svc = SVC(probability=True)
nbc = GaussianNB()
rfc = RandomForestClassifier()

In [ ]:
# Output is sentiment values
y_train = df1['id'].to_numpy()
y_test = df2['id'].to_numpy()

# from sklearn.preprocessing import StandardScaler
# # Remove the mean and scale to unit variance
# scale = StandardScaler()
# # x can be chosen from bag of words, TF-IDF, word2vec
# # x_train = scale.fit_transform(bag_of_words)
# x_train = scale.fit_transform(tf_idf_array)
# # x_train = scale.fit_transform(w2v)

# # x_test = scale.fit_transform(bag_of_words_test)
# x_test = scale.fit_transform(tf_idf_array_test)
# # x_test = scale.fit_transform(w2v_test)

x_train = tf_idf_array
x_test = tf_idf_array_test

In [ ]:
# Pass my training dataset into model.fit() to train my classifiers
lc.fit(x_train, y_train)
nbc.fit(x_train, y_train)
rfc.fit(x_train, y_train)
svc.fit(x_train, y_train)

In [ ]:
# Run model on test dataset
y_lc_predicted = lc.predict(x_test)         # Gives predicted episode number
y_lc_pred_proba = lc.predict_proba(x_test)  # Gives probabilities for each episode number in array form

y_nbc_predicted = nbc.predict(x_test)
y_nbc_pred_proba = nbc.predict_proba(x_test)

y_rfc_predicted = rfc.predict(x_test)
y_rfc_pred_proba = rfc.predict_proba(x_test)

y_svc_predicted = svc.predict(x_test)         # Gives predicted episode number
y_svc_pred_proba = svc.predict_proba(x_test)


In [ ]:
x_test

In [ ]:
print(df1)
print(df2)

In [ ]:
ep_dict = {
  0: "1a: Help Wanted",
  1: "1c: Tea at the Treedome",
  2: "2a: Bubblestand",
  3: "2b: Ripped Pants",
  4: "3a: Jellyfishing",
  5: "3b: Plankton!",
  6: "04a: Naughty Nautical Neighbors",
  7: "04b: Boating School",
  8: "05a: Pizza Delivery",
  9: "05b: Home Sweet Pineapple",
  10: "06a: Mermaid Man and Barnacle Boy",
  11: "06b: Pickles",
  12: "07a: Hall Monitor",
  13: "07b: Jellyfish Jam",
  14: "08a: Sandy's Rocket",
  15: "08b: Squeaky Boots",
  16: "09a: Nature Pants",
  17: "09b: Opposite Day",
  18: "10a: Culture Shock",
  19: "10b: F.U.N",

  20: "11a: MuscleBob BuffPants",
  21: "11b: Squidward the Unfriendly Ghost",
  22: "12a: The Chaperone",
  23: "12b: Employee of the Month",
  24: "13a: Scaredy Pants",
  25: "13b: I Was a Teenage Gary",
  26: "14a: SB-129",
  27: "14b: Karate Choppers",
  28: "15a: Sleepy Time",
  29: "15b: Suds",
  30: "16a: Valentine's Day",
  31: "16b: The Paper",
  32: "17a: Arrgh!",
  33: "17b: Rock Bottom",
  34: "18a: Texas",
  35: "18b: Walking Small",
  36: "19a: Fools in April",
  37: "19b: Neptune's Spatula",
  38: "20a: Hooky",
  39: "20b: Mermaid Man and Barnacle Boy II",



}

In [ ]:
print(y_test)
print(y_lc_predicted)
print(y_lc_pred_proba)
print(classification_report(y_test, y_lc_predicted))

print(y_nbc_predicted)
print(y_nbc_pred_proba)
print(classification_report(y_test, y_nbc_predicted))

print(y_rfc_predicted)
print(y_rfc_pred_proba)
print(classification_report(y_test, y_rfc_predicted))

print(y_test)
print(y_svc_predicted)
print(y_svc_pred_proba)
print(classification_report(y_test, y_svc_predicted))


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def Matrix(trueY, predictedY):
  #
  # Calculate the confusion matrix
  #
  conf_matrix = confusion_matrix(y_true=trueY, y_pred=predictedY)
  #
  # Print the confusion matrix using Matplotlib
  #
  fig, ax = plt.subplots(figsize=(7.5, 7.5))
  ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
  for i in range(conf_matrix.shape[0]):
      for j in range(conf_matrix.shape[1]):
          ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

  plt.xlabel('Predictions', fontsize=18)
  plt.ylabel('Actuals', fontsize=18)
  plt.title('Confusion Matrix', fontsize=18)
  plt.show()

In [ ]:
Matrix(y_test, y_lc_predicted)
Matrix(y_test, y_nbc_predicted)
Matrix(y_test, y_rfc_predicted)
Matrix(y_test, y_svc_predicted)

